### 連線到 MySQL 資料庫

In [1]:
from sqlalchemy import create_engine, text

# 寫成 class 就能建立多個實例，連線到不同資料庫
class MySQL_DB:
    def __init__(self, user: str, password: str, db_name: str, host: str="localhost", port: int=3306) -> None:
        self.host = host
        self.port = str(port)
        self.user = user
        self.password = password # 通常是放在環境變數
        self.db_name = db_name
        self.engine = create_engine(f'mysql+pymysql://{self.user}:{self.password}@{self.host}:{self.port}')
    
    def create_db(self) -> None:
        with self.engine.connect() as connection:
            connection.execute(text(f"CREATE DATABASE IF NOT EXISTS {self.db_name} DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))
            self.engine = create_engine(f'mysql+pymysql://{self.user}:{self.password}@{self.host}:{self.port}/{self.db_name}')
            print(f"資料庫 '{self.db_name}' 建立成功！")    

# 連線到 example_db 資料庫
db = MySQL_DB(user='root', password='example', db_name='example_db')
db.create_db()

資料庫 'example_db' 建立成功！


### 1. 讀取 CSV 然後存入 MySQL

In [3]:
import pandas as pd
from os import path

# 讀取 interview.csv，使用 path.join 來支援不同作業系統
csv_file = path.join(".", "data", "interview.csv")

# 用 pandas 讀成 DataFrame
df = pd.read_csv(csv_file)

# 存入資料庫
df.to_sql('interview', con=db.engine, if_exists='replace', index=False)

# 印出資料庫裡的東西，確定有存進去
print(pd.read_sql('SELECT * FROM interview', con=db.engine))

         id name                 email  age                  city
0         1  蘇思穎     jie35@example.org   23  15594 屏東縣大安街6段131號4樓
1         2  萬俊傑    dliang@example.net   41        553 連江三民街48號之0
2         3  閻怡伶       eli@example.org   54     57672 臺中奇岩巷465號8樓
3         4  李佳樺    guyang@example.org   45        869 台北市民富巷5號之2
4         5  王嘉玲    weifan@example.net   37      15288 關山市光華街4號1樓
...     ...  ...                   ...  ...                   ...
9995   9996  尚惠婷      li60@example.com   32      68703 梅山縣自由巷2號之1
9996   9997  李志宏     leixu@example.net   25         365 中壢林森路4號4樓
9997   9998  邵志豪  duanyong@example.org   37    60090 鳳山縣長春街350號1樓
9998   9999  張怡伶    juan15@example.org   44     983 南投永安路4段469號之6
9999  10000  王詩婷    jungao@example.net   24      200 竹北龍山寺路7段8號之0

[10000 rows x 5 columns]


### 2. 讀取 Mysql 匯入的資料並將使用者的名子末兩碼改成 OO

In [4]:
df = pd.read_sql('SELECT * FROM interview', con=db.engine)
df["name"] = df["name"].str[0] + "OO"
df.to_sql('interview', con=db.engine, if_exists='replace', index=False)

# 印出資料庫裡的東西，確定有存進去
new_df = pd.read_sql('SELECT * FROM interview', con=db.engine)
print(new_df)

         id name                 email  age                  city
0         1  蘇OO     jie35@example.org   23  15594 屏東縣大安街6段131號4樓
1         2  萬OO    dliang@example.net   41        553 連江三民街48號之0
2         3  閻OO       eli@example.org   54     57672 臺中奇岩巷465號8樓
3         4  李OO    guyang@example.org   45        869 台北市民富巷5號之2
4         5  王OO    weifan@example.net   37      15288 關山市光華街4號1樓
...     ...  ...                   ...  ...                   ...
9995   9996  尚OO      li60@example.com   32      68703 梅山縣自由巷2號之1
9996   9997  李OO     leixu@example.net   25         365 中壢林森路4號4樓
9997   9998  邵OO  duanyong@example.org   37    60090 鳳山縣長春街350號1樓
9998   9999  張OO    juan15@example.org   44     983 南投永安路4段469號之6
9999  10000  王OO    jungao@example.net   24      200 竹北龍山寺路7段8號之0

[10000 rows x 5 columns]


### 3. 從 Mysql 撈出 age>30 歲的資料並寫入 google sheet 中

In [4]:
import requests

# 撈出 age > 30 的資料
df = pd.read_sql('SELECT * FROM interview WHERE age > 30', con=db.engine)
print(df)

# 轉成符合 apps script 的格式
rows_dict = df.to_dict(orient='tight')
rows_data = rows_dict['data']
print(rows_data)

# 寫入 google sheet
gas_url = "https://script.google.com/macros/s/AKfycbzUxG_jpSdu-53lEwmjYyR2vzWxFR5yICWZAkRc6MiNrGceNobQOfd9d-W2W8awej6-/exec"

# 將資料分割成 100 筆一組
splitted_rows_data = [rows_data[i:i+100] for i in range(0, len(rows_data), 100)]
print(splitted_rows_data)
i = 0
for data in splitted_rows_data:
    res = requests.post(gas_url, json=data)
    i += 1
    if res.status_code == 200:
        print(f"第 {i} 組資料已成功寫入 Google Sheet，共 {len(splitted_rows_data)} 組")
print("所有資料已成功寫入 Google Sheet")

        id name                 email  age                city
0        2  萬OO    dliang@example.net   41      553 連江三民街48號之0
1        3  閻OO       eli@example.org   54   57672 臺中奇岩巷465號8樓
2        4  李OO    guyang@example.org   45      869 台北市民富巷5號之2
3        5  王OO    weifan@example.net   37    15288 關山市光華街4號1樓
4        6  白OO  jiezhang@example.com   53      789 頭份大仁巷88號3樓
...    ...  ...                   ...  ...                 ...
6900  9992  洪OO    ming28@example.com   37       172 苗栗福安路9號6樓
6901  9995  石OO    sufang@example.com   33     576 太保市民享街73號之7
6902  9996  尚OO      li60@example.com   32    68703 梅山縣自由巷2號之1
6903  9998  邵OO  duanyong@example.org   37  60090 鳳山縣長春街350號1樓
6904  9999  張OO    juan15@example.org   44   983 南投永安路4段469號之6

[6905 rows x 5 columns]
[[2, '萬OO', 'dliang@example.net', 41, '553 連江三民街48號之0'], [3, '閻OO', 'eli@example.org', 54, '57672 臺中奇岩巷465號8樓'], [4, '李OO', 'guyang@example.org', 45, '869 台北市民富巷5號之2'], [5, '王OO', 'weifan@example.net', 37, '15288 關山市光華街

### 4. 從 Mysql 撈出 email為@example.com 的資料並輸出成 excel

In [18]:
import os

df = pd.read_sql("SELECT * FROM interview WHERE email like %s", con=db.engine, params=('%@example.com',))
print(df)
df.to_excel(os.path.join("output", "output.xlsx"), index=False)
print("已成功匯出 output.xlsx")

        id name                  email  age              city
0        6  白OO   jiezhang@example.com   53    789 頭份大仁巷88號3樓
1        8  張OO        bmo@example.com   47  40651 馬公新生街47號7樓
2       14  黃OO     juan31@example.com   53  13226 新營芝山巷22號之0
3       20  陶OO       pyin@example.com   46  634 雲林市建國街731號之5
4       24  石OO  guiying90@example.com   54    431 蘆竹石牌巷78號之1
...    ...  ...                    ...  ...               ...
3317  9991  吳OO     jieyao@example.com   40    295 宜蘭莒光街16號之0
3318  9992  洪OO     ming28@example.com   37     172 苗栗福安路9號6樓
3319  9994  潘OO     minwan@example.com   22  480 雲林縣忠義巷527號之3
3320  9995  石OO     sufang@example.com   33   576 太保市民享街73號之7
3321  9996  尚OO       li60@example.com   32  68703 梅山縣自由巷2號之1

[3322 rows x 5 columns]
已成功匯出 output.xlsx
